In [ ]:
%pip install pymc arviz matplotlib pandas numpy --quiet

In [ ]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Load Brent oil price data
brent = pd.read_csv('BrentOilPrices.csv', parse_dates=['Date'], dayfirst=True)
print(brent.head())
print(brent.info())
print(brent.describe())
print(brent.isnull().sum())

In [ ]:
# Plot the time series
plt.figure(figsize=(15,5))
plt.plot(brent['Date'], brent['Price'])
plt.title('Brent Oil Prices Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD/barrel)')
plt.show()

# Rolling statistics
brent['RollingMean'] = brent['Price'].rolling(window=365).mean()
brent['RollingStd'] = brent['Price'].rolling(window=365).std()

plt.figure(figsize=(15,5))
plt.plot(brent['Date'], brent['Price'], label='Price')
plt.plot(brent['Date'], brent['RollingMean'], label='Rolling Mean (1 year)')
plt.plot(brent['Date'], brent['RollingStd'], label='Rolling Std (1 year)')
plt.legend()
plt.show()

# Stationarity test
result = adfuller(brent['Price'].dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
events = pd.read_csv('MajorEvents.csv', parse_dates=['Start Date'])
print(events)

In [ ]:
import pymc3 as pm
import numpy as np

prices = brent['Price'].values[-2000:]

with pm.Model() as model:
    cp = pm.DiscreteUniform('cp', lower=0, upper=len(prices)-1)
    mu1 = pm.Normal('mu1', mu=np.mean(prices), sd=np.std(prices))
    mu2 = pm.Normal('mu2', mu=np.mean(prices), sd=np.std(prices))
    sigma = pm.HalfNormal('sigma', sd=10)
    mu = pm.math.switch(cp >= np.arange(len(prices)), mu1, mu2)
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=prices)
    trace = pm.sample(2000, tune=1000, cores=2, random_seed=42)

pm.plot_posterior(trace, var_names=['cp'])

## Case 1: Bayesian Inference Calculation (Manual Computation)
First, we update the prior using observed data without Monte Carlo.

In [ ]:
# Given Data
prior_mean = 5      # Prior belief (average oil price increase in $)
prior_std = 2       # Prior uncertainty
likelihood_mean = 8  # Observed data suggests price change of $8 per barrel
likelihood_std = 3   # Observed standard deviation
n = 10   # Number of observations

In [ ]:
# Compute Posterior Mean and Variance
posterior_mean = ( (likelihood_std**2 * prior_mean) + (n * prior_std**2 * likelihood_mean) ) / (likelihood_std**2 + n * prior_std**2)
posterior_variance = (likelihood_std**2 * prior_std**2) / (likelihood_std**2 + n * prior_std**2)
posterior_std = np.sqrt(posterior_variance)


In [ ]:
# Print Results
print(f"Posterior Mean: {posterior_mean:.3f}")
print(f"Posterior Std Dev: {posterior_std:.3f}")

## Case 2: Monte Carlo (MC) Approximation
We use random sampling from the prior and likelihood to approximate the posterior.

In [ ]:
# Monte Carlo Sampling
num_samples = 10000
prior_samples = np.random.normal(prior_mean, prior_std, num_samples)
likelihood_samples = np.random.normal(likelihood_mean, likelihood_std, num_samples)

In [ ]:

# Compute Weighted Posterior Samples
posterior_samples = (prior_samples + likelihood_samples) / 2  # Simple weighting approach

# Plot Results
plt.hist(posterior_samples, bins=50, density=True, alpha=0.6, label="Monte Carlo Approximation")
plt.axvline(posterior_mean, color='r', linestyle='dashed', label="Analytical Posterior Mean")
plt.xlabel("Oil Price Change ($)")
plt.ylabel("Density")
plt.legend()
plt.title("Monte Carlo Approximation of Bayesian Inference")
plt.show()


### Step 3: Markov Chain Monte Carlo (MCMC) with PyMC
We now use MCMC sampling to approximate the true posterior.

python
Copy
Edit


In [ ]:
import pymc as pm
import arviz as az

# Bayesian Modeling with PyMC
with pm.Model() as model:
    theta = pm.Normal("theta", mu=prior_mean, sigma=prior_std)  # Prior
    likelihood = pm.Normal("likelihood", mu=theta, sigma=likelihood_std, observed=[8]*10)  # Observed Data
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)

# Plot Posterior Distribution
az.plot_posterior(trace, var_names=["theta"])
plt.title("Posterior Distribution from MCMC Sampling")
plt.show()

# Display Summary
summary = az.summary(trace, var_names=["theta"])
print(summary)
